In [19]:
import pandas as pd
import os

import matplotlib.pyplot as plt
%matplotlib inline

### add voc

In [76]:
in_dir = 'D:\\Toppan\\2017-11-20 全データ\\処理済(機械ごと)\\elec'
out_dir = os.path.join(in_dir, 'with_voc')

files = [x for x in os.listdir(in_dir) if '.xlsx' in x]

for f in files:
    
    print(f)
    
    exl = pd.ExcelFile(os.path.join(in_dir, f))
    exl_writer = pd.ExcelWriter(os.path.join(out_dir, f))
    
    voc_df = exl.parse(sheet_name='voc', 
              encoding='shift-jis', 
              index_col=0, 
              parse_dates=True)
    
    for sheet in exl.sheet_names:
        
        if sheet in ['voc']: 
            continue
        
        #print(sheet)
        
        df = exl.parse(sheet, 
                       encoding='shift-jis', 
                       index_col=0, 
                       parse_dates=True)
        
        if sheet != 'elec_others':
            df.drop(columns=['計画開始数', '計画完了数'], inplace=True)
        
        if f == '201704010800_elec_per_machine.xlsx':
            df = df.drop([x for x in df.index if x > voc_df.index[-1]])    
        
        def insert_voc(df, name):
            df.insert(0, 'VOC燃料生成量', voc_df['VOC燃料生成量'].values)
            df.insert(0, 'VOC再利用生成量', voc_df['VOC再利用生成量'].values)
            
            df.to_excel(exl_writer, sheet_name=name, encoding='shift-jis')
        
        #その他（総量）	エネルギー棟未取得	工場棟未取得	溶剤回収	空調（総量）
        if sheet == 'elec_others':
            
            df_others_total = df['その他（総量）'].to_frame()
            df_energy_not_got = df['エネルギー棟未取得'].to_frame()
            df_factory_not_got = df['工場棟未取得'].to_frame()
            df_solvent_recoverry = df['溶剤回収'].to_frame()
            df_air_total = df['空調（総量）'].to_frame()
            
            insert_voc(df_others_total, 'その他（総量）')
            insert_voc(df_energy_not_got, 'エネルギー棟未取得')
            insert_voc(df_factory_not_got, '工場棟未取得')
            insert_voc(df_solvent_recoverry, '溶剤回収')
            insert_voc(df_air_total, '空調（総量）')
        else:
            insert_voc(df, sheet)
    exl_writer.save()
        

201611010800_elec_per_machine.xlsx
201612010800_elec_per_machine.xlsx
201701010800_elec_per_machine.xlsx
201702010800_elec_per_machine.xlsx
201703010800_elec_per_machine.xlsx
201704010800_elec_per_machine.xlsx
201705010800_elec_per_machine.xlsx
201706010800_elec_per_machine.xlsx
201707010800_elec_per_machine.xlsx
201708010800_elec_per_machine.xlsx
201709010800_elec_per_machine.xlsx
201710010800_elec_per_machine.xlsx
201711010800_elec_per_machine.xlsx
201712010800_elec_per_machine.xlsx
201801010800_elec_per_machine.xlsx
201802010800_elec_per_machine.xlsx
~$201802010800_elec_per_machine.xlsx


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Toppan\\2017-11-20 全データ\\処理済(機械ごと)\\elec\\~$201802010800_elec_per_machine.xlsx'

In [77]:
in_dir = 'D:\\Toppan\\2017-11-20 全データ\\処理済(機械ごと)\\elec\\with_voc'
out_dir = os.path.join(in_dir, '..','with_voc_vectorized')

### vectorize

In [78]:
def vectorize(fname):

    out_exl = pd.ExcelWriter(os.path.join(out_dir, fname))
    exl = pd.ExcelFile(os.path.join(in_dir, fname))
    
    def concat(df):
        # concat data of current hour and past 2 hours
        df0, df1, df2 = df.iloc[:-2], df.iloc[1:-1], df.iloc[2:]
        
        # change column names
        df0.columns = [x + '-3' for x in df0.columns]
        df1.columns = [x + '-2' for x in df1.columns]
        df2.columns = [x + '-1' for x in df2.columns]
        
        df0.index = df2.index.tolist()
        df1.index = df2.index.tolist()
        
        df012 = pd.concat([df0, df1, df2], axis=1)
        
        # add target
        df012['target'] = df012.iloc[:, -1]
        
        return df012
    
    for name in exl.sheet_names:
            
        # read
        df = exl.parse(name, 
                       index_col=0, 
                       parse_dates=True, 
                       encoding='shift-jis')
        
        # concat
        df012 = concat(df)
        
        # output
        df012.to_excel(out_exl, 
                       sheet_name=name, 
                       encoding='shift-jis')
        
    out_exl.save()

In [79]:
files = os.listdir(in_dir)
for f in files:
    print(f)
    vectorize(f)
print('finished')

201611010800_elec_per_machine.xlsx
201612010800_elec_per_machine.xlsx
201701010800_elec_per_machine.xlsx
201702010800_elec_per_machine.xlsx
201703010800_elec_per_machine.xlsx
201704010800_elec_per_machine.xlsx
201705010800_elec_per_machine.xlsx
201706010800_elec_per_machine.xlsx
201707010800_elec_per_machine.xlsx
201708010800_elec_per_machine.xlsx
201709010800_elec_per_machine.xlsx
201710010800_elec_per_machine.xlsx
201711010800_elec_per_machine.xlsx
201712010800_elec_per_machine.xlsx
201801010800_elec_per_machine.xlsx
201802010800_elec_per_machine.xlsx
finished


### compute total_acc.csv

In [74]:
in_dir = 'D:\\Toppan\\2017-11-20 全データ\\解析結果(機械ごと)\\月間モデル_追加学習あり_全休日除き'
dirs = os.listdir(in_dir)
#dirs = ['17年5月']

for d in dirs:
    
    print(d)
    
    d_path = os.path.join(in_dir, d)
    acc_files = [x for x in os.listdir(d_path) if 'acc' in x and 'total' not in x]
    print(acc_files)
    accs = []
    for f in acc_files:
        acc = pd.read_csv(os.path.join(d_path, f), 
                          encoding='shift-jis', 
                          engine='python', 
                          index_col=0)
        accs.append(acc)
    
    accs = pd.concat(accs)
    accs.to_csv(os.path.join(d_path, 'total_acc.csv'), 
                encoding='shift-jis')

print('finished')

16年12月
['GDNB_acc.csv', 'GE07_acc.csv', 'GE51_acc.csv', 'GE52_acc.csv', 'GE53_acc.csv', 'GL15_acc.csv', 'GL51_acc.csv', 'GL52_acc.csv', 'GP22_acc.csv', 'GP25_acc.csv', 'GP26_acc.csv', 'GP27_acc.csv', 'GP28_acc.csv', 'GP29_acc.csv', 'GP30_acc.csv', 'GP31_acc.csv', 'GP51_acc.csv', 'GP52_acc.csv', 'GP53_acc.csv', 'GT1A_GT1B_acc.csv', '溶剤回収_acc.csv', '空調_acc.csv']
17年10月
['GDNB_acc.csv', 'GE07_acc.csv', 'GE51_acc.csv', 'GE52_acc.csv', 'GE53_acc.csv', 'GL15_acc.csv', 'GL51_acc.csv', 'GL52_acc.csv', 'GP22_acc.csv', 'GP25_acc.csv', 'GP26_acc.csv', 'GP27_acc.csv', 'GP28_acc.csv', 'GP29_acc.csv', 'GP30_acc.csv', 'GP31_acc.csv', 'GP51_acc.csv', 'GP52_acc.csv', 'GP53_acc.csv', 'GT1A_GT1B_acc.csv', '溶剤回収_acc.csv', '空調_acc.csv']
17年11月
['GDNB_acc.csv', 'GE07_acc.csv', 'GE51_acc.csv', 'GE52_acc.csv', 'GE53_acc.csv', 'GL15_acc.csv', 'GL51_acc.csv', 'GL52_acc.csv', 'GP22_acc.csv', 'GP25_acc.csv', 'GP26_acc.csv', 'GP27_acc.csv', 'GP28_acc.csv', 'GP29_acc.csv', 'GP30_acc.csv', 'GP31_acc.csv', 'GP51_acc.

NotADirectoryError: [WinError 267] ディレクトリ名が無効です。: 'D:\\Toppan\\2017-11-20 全データ\\解析結果(機械ごと)\\月間モデル_追加学習あり_全休日除き\\total_acc.csv'